In [1]:
import re
def string_matching_percentage(istr1, istr2, merge_dict={"[QN]":"Q"},ignore_lst="[KCYDE]", default=None):
    str1 = istr1
    str2 = istr2
    if ignore_lst is not None:
        # masking KCYDE
        str1 = re.sub(ignore_lst,'',str1)
        str2 = re.sub(ignore_lst,'',str2)
    if merge_dict is not None:
        # merging
        for k,v in merge_dict.items():
            str1 = re.sub(k,v,str1)
            str2 = re.sub(k,v,str2)
    # calculating matching score
    if len(str1) != len(str2):
        if default is None:
            print("string 1:", str1,"\nstring 2:", str2)
            raise ValueError("Both strings must have the same length for comparison. (You can set default (float) = something to bypass this error)")
        else:
            return default      
    matching_count = sum(1 for a, b in zip(str1, str2) if a == b and a.isalpha() )
    total_length = len(str1)
    
    percentage = (matching_count / total_length) * 100.0
    return percentage

In [2]:
import pandas as pd


all_files = {
    'E-Coli - KCYMR' : '/home/tlp5359/projects/CC_Sequencing_LLM/BCB/inference-results/KCYMR/E-Coli/UniRef100-F1.json',
    'E-Coli - KCYMH' : '/home/tlp5359/projects/CC_Sequencing_LLM/BCB/inference-results/KCYMH/E-Coli/UniRef100-F1.json',
    'E-Coli - KCYMW' : '/home/tlp5359/projects/CC_Sequencing_LLM/BCB/inference-results/KCYMW/E-Coli/UniRef100-F1.json',
    'E-Coli - KCYMS' : '/home/tlp5359/projects/CC_Sequencing_LLM/BCB/inference-results/KCYMS/E-Coli/UniRef100-F1.json',
    'E-Coli - KCYMT' : '/home/tlp5359/projects/CC_Sequencing_LLM/BCB/inference-results/KCYMT/E-Coli/UniRef100-F1.json',
}


from collections import defaultdict
d = defaultdict(list)
for k, v in all_files.items():
    tmp_df = None
    tmp_df = pd.read_json(v)[['Matching Percentage','Matching Percentage full','Masking %','expected','predicted']]
    d['key'].append(k)
    d['Matching Percentage'].append(tmp_df['Matching Percentage'].mean())
    d['Matching Percentage std'].append(tmp_df['Matching Percentage'].std())
    d['Matching Percentage full'].append(tmp_df['Matching Percentage full'].mean())
    d['Matching Percentage full std'].append(tmp_df['Matching Percentage full'].std())
    d['Masking %'].append(tmp_df['Masking %'].mean())
    d['Masking % atd'].append(tmp_df['Masking %'].std())
    pred_seqs = ''.join(list(tmp_df['predicted']))
    real_seqs = ''.join(list(tmp_df['expected']))
    d['token acc'].append(string_matching_percentage(real_seqs,pred_seqs,merge_dict=None, ignore_lst=' '))
    print("Done ",k)
df = pd.DataFrame(d)

Done  E-Coli - KCYMR
Done  E-Coli - KCYMH
Done  E-Coli - KCYMW
Done  E-Coli - KCYMS
Done  E-Coli - KCYMT


In [3]:
df.T

,0,1,2,3,4
key,E-Coli - KCYMR,E-Coli - KCYMH,E-Coli - KCYMW,E-Coli - KCYMS,E-Coli - KCYMT
Matching Percentage,76.283885,75.002202,75.826755,75.220284,75.744692
Matching Percentage std,32.264604,33.251759,32.922903,32.907187,32.534054
Matching Percentage full,80.552479,78.567453,79.123259,79.831905,80.082394
Matching Percentage full std,26.414937,28.456332,28.397342,26.706576,26.666213
Masking %,82.780287,86.299496,86.922493,82.337601,82.882206
Masking % atd,3.725156,3.401726,3.275879,3.693052,3.425201
token acc,80.259758,78.851887,79.517562,78.703869,80.37731


In [4]:
import pandas as pd
from collections import defaultdict

d = defaultdict(list)

for N in [1,2,3]:
    v = f"/home/tlp5359/projects/CC_Sequencing_LLM/BCB/inference-results/KCYM/E-fergusonii-to-others/UniRef100-F{N}.json"
    tmp_df = None
    tmp_df = pd.read_json(v)
    tmp_df = tmp_df[tmp_df['seq_label'] == 'E-fergusonii'][['Matching Percentage','Matching Percentage full','Masking %','expected','predicted']]
#     d['key'].append(k)
    d['Matching Percentage'].append(tmp_df['Matching Percentage'].mean())
    d['Matching Percentage std'].append(tmp_df['Matching Percentage'].std())
    d['Matching Percentage full'].append(tmp_df['Matching Percentage full'].mean())
    d['Matching Percentage full std'].append(tmp_df['Matching Percentage full'].std())
    d['Masking %'].append(tmp_df['Masking %'].mean())
    d['Masking % atd'].append(tmp_df['Masking %'].std())
    pred_seqs = ''.join(list(tmp_df['predicted']))
    real_seqs = ''.join(list(tmp_df['expected']))
    d['token acc'].append(string_matching_percentage(real_seqs,pred_seqs,merge_dict=None, ignore_lst=' '))
    print("Done +1")
df = pd.DataFrame(d)
df

Done +1
Done +1
Done +1


,Matching Percentage,Matching Percentage std,Matching Percentage full,Matching Percentage full std,Masking %,Masking % atd,token acc
0,88.241437,24.547187,89.655381,21.523987,88.794954,2.911488,87.261900
1,88.209964,24.890345,89.617022,21.861004,88.846392,2.901398,86.032138
2,88.549186,24.438183,89.919079,21.460849,88.847115,2.915451,87.197561


In [5]:
df.mean()

Matching Percentage             88.333529
Matching Percentage std         24.625239
Matching Percentage full        89.730494
Matching Percentage full std    21.615280
Masking %                       88.829487
Masking % atd                    2.909446
token acc                       86.830533
dtype: float64

In [6]:
df.std()

Matching Percentage             0.187427
Matching Percentage std         0.235969
Matching Percentage full        0.164442
Matching Percentage full std    0.215132
Masking %                       0.029908
Masking % atd                   0.007246
token acc                       0.692178
dtype: float64

In [16]:
from collections import defaultdict
d = defaultdict(list)

for N in [1,2,3]:
    v = f"/home/tlp5359/projects/CC_Sequencing_LLM/BCB/inference-results/KCYMRHWST/E-fergusonii-to-others/UniRef100-F{N}.json"
    tmp_df = None
    tmp_df = pd.read_json(v)
    tmp_df = tmp_df[tmp_df['seq_label'] == 'E-fergusonii'][['Matching Percentage','Matching Percentage full','Masking %','expected','predicted']]
#     d['key'].append(k)
    d['Matching Percentage'].append(tmp_df['Matching Percentage'].mean())
    d['Matching Percentage std'].append(tmp_df['Matching Percentage'].std())
    d['Matching Percentage full'].append(tmp_df['Matching Percentage full'].mean())
    d['Matching Percentage full std'].append(tmp_df['Matching Percentage full'].std())
    d['Masking %'].append(tmp_df['Masking %'].mean())
    d['Masking % atd'].append(tmp_df['Masking %'].std())
    pred_seqs = ''.join(list(tmp_df['predicted']))
    real_seqs = ''.join(list(tmp_df['expected']))
    d['token acc'].append(string_matching_percentage(real_seqs,pred_seqs,merge_dict=None, ignore_lst=' '))
    print("Done +1")
df = pd.DataFrame(d)
df

Done +1
Done +1
Done +1


,Matching Percentage,Matching Percentage std,Matching Percentage full,Matching Percentage full std,Masking %,Masking % atd,token acc
0,93.551816,18.628099,95.735404,12.263667,67.941439,3.765788,93.179630
1,93.314244,19.122521,95.574566,12.605564,68.014980,3.784950,92.265783
2,93.606861,18.662268,95.760229,12.355723,67.957872,3.866432,93.094767


In [17]:
df.mean()

Matching Percentage             93.490974
Matching Percentage std         18.804296
Matching Percentage full        95.690066
Matching Percentage full std    12.408318
Masking %                       67.971430
Masking % atd                    3.805723
token acc                       92.846727
dtype: float64

In [18]:
df.std()

Matching Percentage             0.155507
Matching Percentage std         0.276120
Matching Percentage full        0.100794
Matching Percentage full std    0.176913
Masking %                       0.038600
Masking % atd                   0.053441
token acc                       0.504898
dtype: float64